In [1]:
from dotenv import load_dotenv
import os

from langchain_community.graphs import Neo4jGraph

# Warning control
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# docker run \
#     --name neo4j \
#     -p 7474:7474 -p 7687:7687 \
#     -d \
#     -e NEO4J_AUTH=neo4j/password \
#     -e NEO4J_PLUGINS=\[\"apoc\"\]  \
#     neo4j:latest

# or you can spin up a demo instance at https://demo.neo4jlabs.com/ or 
# use the docker compose file in this repo

In [70]:
load_dotenv(override=True)
NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
NEO4J_DATABASE = os.getenv('NEO4J_DATABASE')

In [71]:
print(NEO4J_DATABASE)

neo4j


In [72]:
kg = Neo4jGraph(
    url=NEO4J_URI, 
    username=NEO4J_USERNAME, 
    password=NEO4J_PASSWORD, 
    database=NEO4J_DATABASE
)

### Match all nodes in the graph

In [73]:
cypher = """
  MATCH (n) 
  RETURN count(n) AS node_count
  """

In [74]:
result = kg.query(cypher)
result

[{'node_count': 171}]

In [75]:
cypher = """
  MATCH (m:Movie) 
  RETURN count(m) AS movie_count
  """

kg.query(cypher)

[{'movie_count': 38}]

In [42]:
cypher = """
  MATCH (p:Person) 
  RETURN count(p) AS person_count
  """

kg.query(cypher)

[{'person_count': 133}]

In [44]:
cypher = """
  MATCH (tom:Person {name: "Tom Hanks"})
  RETURN tom
  """

kg.query(cypher)

[{'tom': {'born': 1956, 'name': 'Tom Hanks'}}]

In [45]:
cypher = """
  MATCH (cloudAtlas:Movie {title: "Cloud Atlas"})
  RETURN cloudAtlas
  """

kg.query(cypher)

[{'cloudAtlas': {'tagline': 'Everything is connected',
   'votes': 121,
   'title': 'Cloud Atlas',
   'released': 2012}}]

In [46]:
cypher = """
  MATCH (cloudAtlas:Movie {title: "Cloud Atlas"})
  RETURN cloudAtlas.released
  """

kg.query(cypher)

[{'cloudAtlas.released': 2012}]

In [47]:
cypher = """
  MATCH (cloudAtlas:Movie {title: "Cloud Atlas"})
  RETURN cloudAtlas.released, cloudAtlas.tagline
  """

kg.query(cypher)

[{'cloudAtlas.released': 2012,
  'cloudAtlas.tagline': 'Everything is connected'}]

### Conditional Matching

In [48]:
cypher = """
  MATCH (nineties:Movie)
  WHERE nineties.released >= 1990 AND nineties.released < 2000
  RETURN nineties.title, nineties.released
  """

kg.query(cypher)

[{'nineties.title': 'The Matrix', 'nineties.released': 1999},
 {'nineties.title': "The Devil's Advocate", 'nineties.released': 1997},
 {'nineties.title': 'A Few Good Men', 'nineties.released': 1992},
 {'nineties.title': 'As Good as It Gets', 'nineties.released': 1997},
 {'nineties.title': 'What Dreams May Come', 'nineties.released': 1998},
 {'nineties.title': 'Snow Falling on Cedars', 'nineties.released': 1999},
 {'nineties.title': "You've Got Mail", 'nineties.released': 1998},
 {'nineties.title': 'Sleepless in Seattle', 'nineties.released': 1993},
 {'nineties.title': 'Joe Versus the Volcano', 'nineties.released': 1990},
 {'nineties.title': 'When Harry Met Sally', 'nineties.released': 1998},
 {'nineties.title': 'That Thing You Do', 'nineties.released': 1996},
 {'nineties.title': 'The Birdcage', 'nineties.released': 1996},
 {'nineties.title': 'Unforgiven', 'nineties.released': 1992},
 {'nineties.title': 'Johnny Mnemonic', 'nineties.released': 1995},
 {'nineties.title': 'The Green Mile',

In [50]:
cypher = """
  MATCH (actor:Person)-[:ACTED_IN]->(movie:Movie)
  RETURN actor.name, movie.title LIMIT 10
  """

kg.query(cypher)

[{'actor.name': 'Emil Eifrem', 'movie.title': 'The Matrix'},
 {'actor.name': 'Hugo Weaving', 'movie.title': 'The Matrix'},
 {'actor.name': 'Laurence Fishburne', 'movie.title': 'The Matrix'},
 {'actor.name': 'Carrie-Anne Moss', 'movie.title': 'The Matrix'},
 {'actor.name': 'Keanu Reeves', 'movie.title': 'The Matrix'},
 {'actor.name': 'Hugo Weaving', 'movie.title': 'The Matrix Reloaded'},
 {'actor.name': 'Laurence Fishburne', 'movie.title': 'The Matrix Reloaded'},
 {'actor.name': 'Carrie-Anne Moss', 'movie.title': 'The Matrix Reloaded'},
 {'actor.name': 'Keanu Reeves', 'movie.title': 'The Matrix Reloaded'},
 {'actor.name': 'Hugo Weaving', 'movie.title': 'The Matrix Revolutions'}]

In [51]:
cypher = """
  MATCH (tom:Person {name: "Tom Hanks"})-[:ACTED_IN]->(tomHanksMovies:Movie)
  RETURN tom.name, tomHanksMovies.title 
  """

kg.query(cypher)

[{'tom.name': 'Tom Hanks', 'tomHanksMovies.title': 'Apollo 13'},
 {'tom.name': 'Tom Hanks', 'tomHanksMovies.title': "You've Got Mail"},
 {'tom.name': 'Tom Hanks', 'tomHanksMovies.title': 'A League of Their Own'},
 {'tom.name': 'Tom Hanks', 'tomHanksMovies.title': 'Joe Versus the Volcano'},
 {'tom.name': 'Tom Hanks', 'tomHanksMovies.title': 'That Thing You Do'},
 {'tom.name': 'Tom Hanks', 'tomHanksMovies.title': 'The Da Vinci Code'},
 {'tom.name': 'Tom Hanks', 'tomHanksMovies.title': 'Cloud Atlas'},
 {'tom.name': 'Tom Hanks', 'tomHanksMovies.title': 'Cast Away'},
 {'tom.name': 'Tom Hanks', 'tomHanksMovies.title': 'The Green Mile'},
 {'tom.name': 'Tom Hanks', 'tomHanksMovies.title': 'Sleepless in Seattle'},
 {'tom.name': 'Tom Hanks', 'tomHanksMovies.title': 'The Polar Express'},
 {'tom.name': 'Tom Hanks', 'tomHanksMovies.title': "Charlie Wilson's War"}]

In [52]:
cypher = """
  MATCH (tom:Person {name: "Tom Hanks"})-[:ACTED_IN]-> (m) <-[:ACTED_IN]-(coActors)
  RETURN coActors.name, m.title 
  """

kg.query(cypher)

[#F3BB]  _: <CONNECTION> error: Failed to read from defunct connection IPv4Address(('demo.neo4jlabs.com', 7687)) (ResolvedIPv4Address(('3.235.178.21', 7687))): OSError('No data')
Transaction failed and will be retried in 1.0084800891234182s (Failed to read from defunct connection IPv4Address(('demo.neo4jlabs.com', 7687)) (ResolvedIPv4Address(('3.235.178.21', 7687))))


[{'coActors.name': 'Ed Harris', 'm.title': 'Apollo 13'},
 {'coActors.name': 'Gary Sinise', 'm.title': 'Apollo 13'},
 {'coActors.name': 'Kevin Bacon', 'm.title': 'Apollo 13'},
 {'coActors.name': 'Bill Paxton', 'm.title': 'Apollo 13'},
 {'coActors.name': 'Parker Posey', 'm.title': "You've Got Mail"},
 {'coActors.name': 'Greg Kinnear', 'm.title': "You've Got Mail"},
 {'coActors.name': 'Meg Ryan', 'm.title': "You've Got Mail"},
 {'coActors.name': 'Steve Zahn', 'm.title': "You've Got Mail"},
 {'coActors.name': 'Dave Chappelle', 'm.title': "You've Got Mail"},
 {'coActors.name': 'Madonna', 'm.title': 'A League of Their Own'},
 {'coActors.name': "Rosie O'Donnell", 'm.title': 'A League of Their Own'},
 {'coActors.name': 'Geena Davis', 'm.title': 'A League of Their Own'},
 {'coActors.name': 'Bill Paxton', 'm.title': 'A League of Their Own'},
 {'coActors.name': 'Lori Petty', 'm.title': 'A League of Their Own'},
 {'coActors.name': 'Nathan Lane', 'm.title': 'Joe Versus the Volcano'},
 {'coActors.na

### Delete data from the graph

In [76]:
cypher = """
  MATCH (emil:Person {name: "Emil Eifrem"})-[actedIn:ACTED_IN]->(movie:Movie)
  DELETE actedIn
  """

kg.query(cypher) # check for permissions

[]

### Add data to the graph

In [77]:
cypher = """
CREATE (rach:Person {name: "Rach"})
RETURN rach
"""

kg.query(cypher)

[{'rach': {'name': 'Rach'}}]

In [78]:
cypher = """
MATCH (rach:Person {name:"Rach"}), (emil:Person {name:"Emil Eifrem"})
MERGE (rach)-[hasRelationship:WORKS_WITH]->(emil)
RETURN rach, hasRelationship, emil
"""
kg.query(cypher)

[{'rach': {'name': 'Rach'},
  'hasRelationship': ({'name': 'Rach'},
   'WORKS_WITH',
   {'born': 1978, 'name': 'Emil Eifrem'}),
  'emil': {'born': 1978, 'name': 'Emil Eifrem'}}]